In [ ]:
import numpy as np

In [ ]:
class KalmanFilter(object):
    def __init__(self, A = None, B = None, C = None, Gamma = None, Sigma = None, P = None, z0 = None):

        if(A is None or C is None):
            raise ValueError("Set proper system dynamics.")

        self.n = A.shape[1]
        self.k = C.shape[1]

        self.A = A
        self.C = C
        self.B = 0 if B is None else B
        self.Gamma = np.eye(self.n) if Gamma is None else Gamma
        self.Sigma = np.eye(self.n) if Sigma is None else Sigma
        self.P = np.eye(self.n) if P is None else P
        self.z = np.zeros((self.n, 1)) if z0 is None else z0

    def predict(self, u = 0):
        self.z = np.dot(self.A, self.z) + np.dot(self.B, u)
        self.P = np.dot(np.dot(self.A, self.P), self.A.T) + self.Gamma
        return self.z

    def update(self, z):
        y = z - np.dot(self.C, self.z)
        
        S = self.Sigma + np.dot(self.C, np.dot(self.P, self.C.T))
        K = np.dot(np.dot(self.P, self.C.T), np.linalg.inv(S))
        self.z = self.z + np.dot(K, y)

        I = np.eye(self.n)
        # self.P = np.dot(I - np.dot(K, self.C), self.P)
        self.P = np.dot(np.dot(I - np.dot(K, self.C), self.P), 
        	(I - np.dot(K, self.C)).T) + np.dot(np.dot(K, self.Sigma), K.T)

def example():
	dt = 1.0/60
	A = np.array([[1, dt, 0], [0, 1, dt], [0, 0, 1]])
	C = np.array([1, 0, 0]).reshape(1, 3)
	Gamma = np.array([[0.05, 0.05, 0.0], [0.05, 0.05, 0.0], [0.0, 0.0, 0.0]])
	Sigma = np.array([0.5]).reshape(1, 1)

	x = np.linspace(-10, 10, 100)
	measurements = - (x**2 + 2*x - 2)  + np.random.normal(0, 2, 100)

	kf = KalmanFilter(A = A, C = C, Gamma = Gamma, Sigma = Sigma)
	predictions = []

	for z in measurements:
		predictions.append(np.dot(C,  kf.predict())[0])
		kf.update(z)

	import matplotlib.pyplot as plt
	plt.plot(range(len(measurements)), measurements, label = 'Measurements')
	plt.plot(range(len(predictions)), np.array(predictions), label = 'Kalman Filter Prediction')
	plt.legend()
	plt.show()

if __name__ == '__main__':
    example()